In [2]:
import pandas as pd
import pandasql as ps

# Корабли

Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](images/ships.png)

In [3]:
classes = pd.read_csv('data/classes.csv', index_col='class')
classes.head()

,type,country,guns,bore,displacement
class,,,,,
Bismarck,bb,Germany,8,15.0,42000
Iowa,bb,USA,9,16.0,46000
Kongo,bc,Japan,8,14.0,32000
North Carolina,bb,USA,12,16.0,37000
Renown,bc,Gt.Britain,6,15.0,32000


In [4]:
ships = pd.read_csv('data/ships.csv', index_col='name')
ships.head()

,class,launched
name,,
California,Tennessee,1921
Haruna,Kongo,1916
Hiei,Kongo,1914
Iowa,Iowa,1943
Kirishima,Kongo,1915


In [5]:
outcomes = pd.read_csv('data/outcomes.csv', index_col=['ship', 'battle'])
outcomes.head()

,,result
ship,battle,
Bismarck,North Atlantic,sunk
California,Surigao Strait,OK
Duke of York,North Cape,OK
Fuso,Surigao Strait,sunk
Hood,North Atlantic,sunk


In [6]:
battles = pd.read_csv('data/battles.csv', index_col='name', parse_dates=['date'])
battles.head()

,date
name,
Guadalcanal,1942-11-15
North Atlantic,1941-05-25
North Cape,1943-12-26
Surigao Strait,1944-10-25


# Задача 1.

Найдите страны, имевшие когда-либо классы обычных боевых кораблей ('bb') и имевшие когда-либо классы крейсеров ('bc').

![](images/ships.png)

### SQL

In [7]:
query = '''
    select country
    from classes
    group by country
    having count(distinct type) = 2
'''
data = ps.sqldf(query)
data

,country
0,Gt.Britain
1,Japan


# Задача 2.

Для каждого класса определите год, когда был спущен на воду первый корабль из этого класса.

Здесь можно предобрабатывать внутреннюю таблицу заранее, или группировать таблицы уже после их слияния.

![](images/ships.png)

### SQL

In [8]:
query = '''
    select c.class, t.y
    from classes c
    left join (
        select class, min(launched) as y
        from ships
        group by class
    ) as t on c.class = t.class
'''
data = ps.sqldf(query)
data

,class,y
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


In [9]:
query = '''
    select classes.class, min(launched)
    from classes left join ships
    on classes.class = ships.class
    group by classes.class
    '''
data = ps.sqldf(query)
data

,class,min(launched)
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


# Самолёты

Таблица company содержит идентификатор и название компании, осуществляющей перевозку пассажиров. Таблица trip содержит информацию о рейсах: номер рейса, идентификатор компании, тип самолета, город отправления, город прибытия, время отправления и время прибытия. Таблица passenger содержит идентификатор и имя пассажира. Таблица pass_in_trip содержит информацию о полетах: номер рейса, дата вылета (день), идентификатор пассажира и место, на котором он сидел во время полета.


![](images/planes.png)

In [10]:
trip = pd.read_csv('data/trip.csv', index_col='trip_no',
    parse_dates=['time_out', 'time_in'], infer_datetime_format=True)
trip.head()

,id_comp,plane,town_from,town_to,time_out,time_in
trip_no,,,,,,
1100,4,Boeing,Rostov,Paris,1900-01-01 14:30:00,1900-01-01 17:50:00
1101,4,Boeing,Paris,Rostov,1900-01-01 08:12:00,1900-01-01 11:45:00
1123,3,TU-154,Rostov,Vladivostok,1900-01-01 16:20:00,1900-01-02 03:40:00
1124,3,TU-154,Vladivostok,Rostov,1900-01-01 09:00:00,1900-01-01 19:50:00
1145,2,IL-86,Moscow,Rostov,1900-01-01 09:35:00,1900-01-01 11:23:00


In [11]:
pass_in_trip = pd.read_csv('data/pass_in_trip.csv',
    infer_datetime_format=True, parse_dates=['date'])
pass_in_trip.head()

,trip_no,date,id_psg,place
0,1100,2003-04-29,1,1a
1,1123,2003-04-05,3,2a
2,1123,2003-04-08,1,4c
3,1123,2003-04-08,6,4b
4,1124,2003-04-02,2,2d


In [12]:
company = pd.read_csv('data/company.csv', index_col='id_comp')
company.head()

,name
id_comp,
1,Don_avia
2,Aeroflot
3,Dale_avia
4,air_France
5,British_AW


In [13]:
passenger = pd.read_csv('data/passenger.csv', index_col='id_psg')
passenger.head()

,name
id_psg,
1,Bruce Willis
2,George Clooney
3,Kevin Costner
4,Donald Sutherland
5,Jennifer Lopez


# Задача 3.

Определить имена пассажиров, когда-либо летевших на одном и том же месте более одного раза.

Существует множество способов решения задачи как с помощью join, так и с помощью where in.

![](images/planes.png)

### SQL

In [14]:
query = '''
    select name from passenger
    join 
        (select distinct id_psg from pass_in_trip
        group by id_psg, place
        having count() > 1 ) as x 
    on x.id_psg = passenger.id_psg
    '''
ps.sqldf(query)

,name
0,Bruce Willis
1,Nikole Kidman
2,Mullah Omar


In [15]:
query = '''
    select name from passenger
    where id_psg in
        (select distinct id_psg from pass_in_trip
        group by id_psg, place
        having count() > 1)
'''
data = ps.sqldf(query)
data

,name
0,Bruce Willis
1,Nikole Kidman
2,Mullah Omar


In [16]:
query = '''
    select
        distinct name
    from passenger join pass_in_trip
    on passenger.id_psg = pass_in_trip.id_psg
    
    group by passenger.id_psg, pass_in_trip.place
    having count(*) > 1 
    '''
ps.sqldf(query)

,name
0,Bruce Willis
1,Nikole Kidman
2,Mullah Omar


# Задача 4.

Для каждой авиакомпании определить:

* количество выполненных перелетов;
* число использованных типов самолетов;
* количество перевезенных различных пассажиров;
* общее число перевезенных компанией пассажиров.

В данной задаче существует потенциально два правильных ответа в зависимости от того, будем ли мы учитывать самолёты компании, на которых никто никогда не летал, и что мы подразумеваем под "использованным" самолётом. Здесь засчитываются оба варианта.

![](images/planes.png)

### SQL

In [17]:
query = '''
    select name,
        count(distinct trip.trip_no),
        count(distinct plane),
        count(distinct id_psg),
        count(*)
    from company, pass_in_trip, trip
    where company.id_comp=trip.id_comp and
        trip.trip_no=pass_in_trip.trip_no
    group by company.id_comp,name
    '''
data = ps.sqldf(query)
data

,name,count(distinct trip.trip_no),count(distinct plane),count(distinct id_psg),count(*)
0,Don_avia,4,1,6,9
1,Aeroflot,1,1,2,2
2,Dale_avia,2,1,4,4
3,air_France,1,1,1,1
4,British_AW,6,1,6,16


In [18]:
query = '''
    select 
        id_comp,
        count() as count_trips,
        count(distinct plane) as count_planes
    from trip
    group by id_comp
'''
ps.sqldf(query)

,id_comp,count_trips,count_planes
0,1,6,2
1,2,2,1
2,3,2,1
3,4,2,1
4,5,10,1


In [19]:
query = '''
    select 
        id_comp,
        count(distinct id_psg) as n_unique_passengers,
        count(id_psg) as n_passengers
    from trip join pass_in_trip
    on trip.trip_no = pass_in_trip.trip_no
    group by id_comp
'''
ps.sqldf(query)

,id_comp,n_unique_passengers,n_passengers
0,1,6,9
1,2,2,2
2,3,4,4
3,4,1,1
4,5,6,16


# Задача 5.
Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов.
Замечания.
* A - B и B - A считать разными маршрутами.
* Использовать только таблицу Trip 

Здесь можно привести два эквивалентных решения с функциями join и where.

![](images/planes.png)

### SQL

In [34]:
query = '''
    select count(*) from

    (select count(*) as count_pathes from trip
    group by town_from, town_to) as a 
    
    join
    
    (select count(*) as max_pathes from trip
    group by town_from, town_to
    order by max_pathes desc
    limit 1) as b
    
    on count_pathes = max_pathes
    
'''
ps.sqldf(query)

,count(*)
0,4


In [35]:
query = '''
    select count(*) from

    (select count(*) as count_pathes from trip
    group by town_from, town_to) as a 
    
    where count_pathes =
        (select count(*) as max_pathes from trip
        group by town_from, town_to
        order by max_pathes desc
        limit 1)
        
'''
ps.sqldf(query)

,count(*)
0,4


# Задача 6.
Определить номера пассажиров, которые больше других времени провели в полетах.
Вывести имена и общее время в минутах, проведенное в полетах

![](images/planes.png)

### SQL

In [21]:
query = '''
    
    /* your code here
    '''
data = ps.sqldf(query)
data